In [12]:
print("Hello World")

Hello World


In [13]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
GITHUB_TOKEN = "YOUR_GITHUB_TOKEN_HERE"

HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

In [17]:
url = "https://api.github.com/search/repositories"
params = {
    "q": "language:python",
    "sort": "stars",
    "order": "desc",
    "per_page": 50
}
try:
    response = requests.get(url, headers=HEADERS, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()
    if data:
        print("Data fetched successfully")

except requests.exceptions.Timeout:
    raise Exception("Request timed out")

except requests.exceptions.HTTPError as e:
    raise Exception(f"HTTP error occurred: {e}")

except requests.exceptions.RequestException as e:
    raise Exception(f"Request failed: {e}")

Data fetched successfully


In [18]:
items = data.get("items", [])
len(items)

50

In [19]:
df = pd.json_normalize(items)
df.head()

,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,license.name,license.spdx_id,license.url,license.node_id,permissions.admin,permissions.maintain,permissions.push,permissions.triage,permissions.pull,license
0,54346799,MDEwOlJlcG9zaXRvcnk1NDM0Njc5OQ==,public-apis,public-apis/public-apis,False,https://github.com/public-apis/public-apis,A collective list of free APIs,False,https://api.github.com/repos/public-apis/publi...,https://api.github.com/repos/public-apis/publi...,...,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,False,False,True,NaN
1,13491895,MDEwOlJlcG9zaXRvcnkxMzQ5MTg5NQ==,free-programming-books,EbookFoundation/free-programming-books,False,https://github.com/EbookFoundation/free-progra...,:books: Freely available programming books,False,https://api.github.com/repos/EbookFoundation/f...,https://api.github.com/repos/EbookFoundation/f...,...,Creative Commons Attribution 4.0 International,CC-BY-4.0,https://api.github.com/licenses/cc-by-4.0,MDc6TGljZW5zZTI1,False,False,False,False,True,NaN
2,83222441,MDEwOlJlcG9zaXRvcnk4MzIyMjQ0MQ==,system-design-primer,donnemartin/system-design-primer,False,https://github.com/donnemartin/system-design-p...,Learn how to design large-scale systems. Prep ...,False,https://api.github.com/repos/donnemartin/syste...,https://api.github.com/repos/donnemartin/syste...,...,Other,NOASSERTION,None,MDc6TGljZW5zZTA=,False,False,False,False,True,NaN
3,21289110,MDEwOlJlcG9zaXRvcnkyMTI4OTExMA==,awesome-python,vinta/awesome-python,False,https://github.com/vinta/awesome-python,An opinionated list of awesome Python framewor...,False,https://api.github.com/repos/vinta/awesome-python,https://api.github.com/repos/vinta/awesome-pyt...,...,Other,NOASSERTION,None,MDc6TGljZW5zZTA=,False,False,False,False,True,NaN
4,63476337,MDEwOlJlcG9zaXRvcnk2MzQ3NjMzNw==,Python,TheAlgorithms/Python,False,https://github.com/TheAlgorithms/Python,All Algorithms implemented in Python,False,https://api.github.com/repos/TheAlgorithms/Python,https://api.github.com/repos/TheAlgorithms/Pyt...,...,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,False,False,True,NaN


In [20]:
df.isna().sum()

id                       0
node_id                  0
name                     0
full_name                0
private                  0
                        ..
permissions.maintain     0
permissions.push         0
permissions.triage       0
permissions.pull         0
license                 50
Length: 108, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               50 non-null     object
 1   full_name          50 non-null     object
 2   owner.login        50 non-null     object
 3   stargazers_count   50 non-null     int64 
 4   forks_count        50 non-null     int64 
 5   open_issues_count  50 non-null     int64 
 6   language           50 non-null     object
 7   created_at         50 non-null     object
 8   updated_at         50 non-null     object
 9   license.name       47 non-null     object
 10  html_url           50 non-null     object
dtypes: int64(3), object(8)
memory usage: 4.4+ KB


In [21]:
columns = [
    "name",
    "full_name",
    "owner.login",
    "stargazers_count",
    "forks_count",
    "open_issues_count",
    "language",
    "created_at",
    "updated_at",
    "license.name",
    "html_url"
]

df = df[columns]
df.head()


,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,383956,40976,747,Python,2016-03-20T23:49:42Z,2025-12-13T20:31:17Z,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378756,65621,203,Python,2013-10-11T06:50:37Z,2025-12-13T20:14:31Z,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329266,53619,517,Python,2017-02-26T16:15:28Z,2025-12-13T20:24:02Z,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273632,26898,526,Python,2014-06-27T21:00:06Z,2025-12-13T20:30:36Z,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214750,49604,792,Python,2016-07-16T09:44:01Z,2025-12-13T20:19:33Z,MIT License,https://github.com/TheAlgorithms/Python


In [ ]:
df_clean = df.dropna(subset=["language"])

In [ ]:
df_clean["license.name"] = df_clean["license.name"].fillna("No License")